In [1]:
import psycopg2
from database import databaseconn
from faker import Faker
import database as creds
from sqlalchemy import create_engine
fake = Faker()
import pandas as pd

In [2]:
engine = create_engine('postgresql://'+creds.username+':'+creds.password+'@'+creds.host+':'+creds.port+'/'+creds.dbname)


In [3]:
def getListDictRs(cur):
    desc = cur.description
    return [dict(zip([col[0] for col in desc], row)) for row in cur.fetchall()]


In [4]:
def insert_owner(name):
    try:
        conn = databaseconn()
        query = """
                    insert into owner(name) values('%s')
                """ % (name)
        with conn, conn.cursor() as cur:  # start a transaction and create a cursor
            cur.execute(query)
            res = cur.fetchone()
            return res
    except psycopg2.DatabaseError as error:
        conn.rollback()
    


In [5]:
# def get_owner_list():
#     query = """
#                 select id, name from owner
#             """
#     cur.execute(query)
#     data = getListDictRs(cur)
#     return None

In [6]:
total_names = 5
devices = 2

In [7]:
name_list = []

In [8]:
def get_name_list(count):
    name_list = []
    for c in range(count):
        name = fake.name()
        while name not in name_list:
           name_list.append(name)
    return name_list

In [17]:
names = get_name_list(10)
names_df=pd.DataFrame([{'name':n} for n in names])
names_df.to_sql("owner", engine, if_exists="append",index=False)


## Get created data from table

In [18]:
names_df = pd.read_sql(
    "SELECT * FROM owner",
    con=engine
)

In [19]:
name_list = names_df.to_dict('records')

In [20]:
device_list = []
for name in name_list:
    for device in range(1, devices+1):
        devide_id = f"{name['name'][:2]}000{device}"
        device_list.append({'owner':name['id'], 'device':devide_id})



In [21]:
device_df = pd.DataFrame(device_list)

In [22]:
device_df.head(2)

,owner,device
0,99,Jo0001
1,99,Jo0002


In [23]:
device_df.to_sql("owner_device", engine, if_exists="append",index=False)

In [25]:
devices = device_df.to_dict('records')

In [27]:
device_df = pd.read_sql(
    "SELECT * FROM owner_device",
    con=engine
)

In [28]:
device_df.head()

,id,owner,device
0,1,99,Jo0001
1,2,99,Jo0002
2,3,100,Th0001
3,4,100,Th0002
4,5,101,Mr0001


In [29]:
devices = device_df.to_dict('records')

In [30]:
for device in devices:


SyntaxError: unexpected EOF while parsing (<ipython-input-30-666fa1a59910>, line 1)

In [38]:
try:
    total_companies = int(input("Enter the total number of companies: "))
    device_per_company = int(
        input("Enter the number of devices to be generated for each company: ")
    )
    # data generation for day, month or year
    print("\n")
    print("***** Interval type can be 'day', 'month', or 'year' *****")
    interval_type = str(input("Enter the interval type: "))
    while interval_type not in ["day", "month", "year"]:
        print("Enter value only of 'day' or 'month' or 'year'!!! ")
        interval_type = str(input("Enter the interval type: "))
    # How many length based on above interval_type
    interval_length = int(input("Enter the interval length(only numeric): "))
    # Interval Period - hour, minute, day
    print("\n")
    print("***** Interval period can be only 'hour', 'minute', or 'day' *****")
    interval_period = str(input("Enter interval period: "))
    while interval_period not in ["hour", "minute", "day"]:
        print("Enter value only of 'hour' or 'minute' or 'day'!!! ")
        interval_period = str(input("Enter interval period: "))
        
    # duration between each data- based on above interval period
    interval_duration = input("Enter interval duration(only numeric): ")
    # if interval_type is year and interval_length is 2 -data will be generated for 2 years
    series_interval_length = f"{interval_length} {interval_type}"
    # if interval_period is hour and interval_duration is 3 -data will be generated for every 3 hours
    series_interval_duration = f"{interval_duration} {interval_period}"

except ValueError:
    print("Enter only numbers !!! ")
    sys.exit()




***** Interval type can be 'day', 'month', or 'year' *****
Enter value only of 'day' or 'month' or 'year'!!! 


***** Interval period can be only 'hour', 'minute', or 'day' *****


In [48]:
for data in devices:
    conn = databaseconn()
    query = """ INSERT INTO device_data (datetime, device_id, temperature, humidity)
                SELECT
                datetime,
                device_id,
                random()*40 AS temperature,
                random()*40 As humidity
                FROM generate_series(now() - interval '%s', now(), interval '%s') AS g1(datetime),
                generate_series(%s,%s) AS g2(device_id)
            """% (series_interval_length, series_interval_duration, data["id"], data["id"])
    with conn, conn.cursor() as cur:
        cur.execute(query)
    print(f"Data inserted for {data['device']}")
    

Data inserted for Jo0001
Data inserted for Jo0002
Data inserted for Th0001
Data inserted for Th0002
Data inserted for Mr0001
Data inserted for Mr0002
Data inserted for Jo0001
Data inserted for Jo0002
Data inserted for Me0001
Data inserted for Me0002
Data inserted for Dr0001
Data inserted for Dr0002
Data inserted for Lo0001
Data inserted for Lo0002
Data inserted for Ma0001
Data inserted for Ma0002
Data inserted for Sh0001
Data inserted for Sh0002
Data inserted for Ed0001
Data inserted for Ed0002
